In [8]:
import requests
from bs4 import BeautifulSoup

In [9]:
url = "https://www.aljazeera.com/where/mozambique/"
def getHTMLdocument(url):
    response = requests.get(url)
    return response.text

aljazeera_paper = getHTMLdocument(url)
print(aljazeera_paper)

<!doctype html><html lang="en" dir="ltr" class="theme-aje archipelago-theme-aje"><head><meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no"/><meta http-equiv="Content-Type" content="text/html;charset=utf-8"><link rel="shortcut icon" href="/favicon_aje.ico"><title data-rh="true">Mozambique | Today&#x27;s latest from Al Jazeera</title>
						<meta data-rh="true" name="title" content="Mozambique | Today&#x27;s latest from Al Jazeera" data-reactroot=""/><meta data-rh="true" name="description" content="Stay on top of Mozambique latest developments on the ground with Al Jazeera’s fact-based news, exclusive video footage, photos and updated maps." data-reactroot=""/><meta data-rh="true" name="robots" content="max-image-preview:large" data-reactroot=""/>
						<link data-rh="true" rel="canonical" href="https://www.aljazeera.com/where/mozambique/" data-reactroot=""/>
						<script data-rh="true" type="application/ld+json" data-reactroot="">


In [10]:
base_url = "https://www.aljazeera.com"
soup = BeautifulSoup(aljazeera_paper, 'html.parser')
all_articles = soup.find_all("a", {"class": "u-clickable-card__link"})

article_links = [base_url + article['href'] for article in all_articles]
print(article_links)

['https://www.aljazeera.com/news/2022/5/23/floods-hit-south-africas-kwazulu-natal-province-again', 'https://www.aljazeera.com/news/2022/3/18/mozambique-cyclone-gombe-death-toll-rises-to-53', 'https://www.aljazeera.com/news/2022/3/4/mozambique-announces-new-prime-minister-and-finance-minister', 'https://www.aljazeera.com/economy/2022/3/1/analysis-can-african-gas-replace-russian-supplies-to-europe', 'https://www.aljazeera.com/news/2022/1/27/at-least-70-dead-from-tropical-storm-ana-in-southern-africa', 'https://www.aljazeera.com/news/2022/1/12/southern-africa-bloc-sadc-extends-mozambique-mission', 'https://www.aljazeera.com/program/start-here/2021/10/3/climate-change-and-famine-start-here', 'https://www.aljazeera.com/news/2021/9/24/in-mozambique-kagame-says-rwandan-troops-presence-to-continue', 'https://www.aljazeera.com/news/2021/8/8/rwanda-mozambique-forces-recapture-port-city-from-rebels', 'https://www.aljazeera.com/news/2021/7/10/rwanda-deploys-1000-soldiers-to-mozambique-cabo-delgado

In [11]:
from newspaper import Article
import json
from tqdm import tqdm

article_collections = []

# article_url = article_links[0]
# article = Article(article_url)
# article.download()
# article.parse()
# article_soup = BeautifulSoup(article.html, 'html.parser')
# article_soup.find("p", {"class": "article__subhead"}).find('em').string

for article_url in tqdm(article_links):
    article = Article(article_url)
    article.download()
    article.parse()
    article_soup = BeautifulSoup(article.html, 'html.parser')
    subtitle = ''
    try:
        subtitle =  article_soup.find("p", {"class": "article__subhead"}).find('em').string
    except:
        pass
    article_collections.append(
        { 'title': article.title, 'subtitle': subtitle, 'content': article.text }
    )

# print(len(article_collections))
with open('data.json', 'w') as f:
    json.dump(article_collections, f)

# json.dumps(article_collections)

100%|███████████████████████████████████████████████████████| 14/14 [00:09<00:00,  1.49it/s]


# Sentiment Analysis

In [51]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /home/rohan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/rohan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/rohan/nltk_data...


True

In [71]:
import json
with open('data.json') as f:
    data = json.load(f)

print(data)

[{'title': 'Floods hit South Africa’s KwaZulu-Natal province again', 'subtitle': 'The floods are happening only a month after torrential rains killed 435 people in the KwaZulu Natal province.', 'content': 'Heavy rains in South Africa have forced hundreds of people to flee their homes on the east coast, according to officials, only a month after the worst floods in living memory killed more than 400.\n\nThe floods over the weekend hit infrastructure, roads, bridges and damaged buildings, mainly in the port city of Durban, the capital of KwaZulu-Natal province and home to some 3.9 million people.\n\n“Some areas are inaccessible and have become islands at this stage,” KwaZulu-Natal Premier Sihle Zikalala said during a news conference on Sunday.\n\nNo fatalities were immediately recorded, but Zikalala added: “We are yet to receive a full comprehensive report on the impact of these latest heavy rains, but as reports come in a worrying picture is however emerging.”\n\nIn April, torrential ra

In [72]:
stopwords = nltk.corpus.stopwords.words("english")

In [73]:
import copy

cleaned_data = copy.deepcopy(data)
preprocessed_data = []
for each in cleaned_data:
    each['content'] = nltk.word_tokenize(each['content'])
    each['content'] = [w for w in each['content'] if w.lower() not in stopwords]
    each['content'] = [w for w in each['content'] if w.lower() not in [",", ".", "(", ")", "”", "[", "]", "“", "’", ":", "–"]]
    preprocessed_data.append(" ".join([w.lower() for w in each['content']]))

print(preprocessed_data)
print(len(preprocessed_data))

['heavy rains south africa forced hundreds people flee homes east coast according officials month worst floods living memory killed 400 floods weekend hit infrastructure roads bridges damaged buildings mainly port city durban capital kwazulu-natal province home 3.9 million people areas inaccessible become islands stage kwazulu-natal premier sihle zikalala said news conference sunday fatalities immediately recorded zikalala added yet receive full comprehensive report impact latest heavy rains reports come worrying picture however emerging. april torrential rain led floods landslides death 435 people kwazulu-natal authorities said survivors left without drinking water nearly two weeks repairs still ongoing floods caused damage worth hundreds millions dollars sunday parts road near umdloti beach resort north durban washed away leaving gaping holes cars balanced edge afp news agency reported flooding started late yesterday saturday afternoon result collapse road get one local resident kevi

In [77]:
from nltk.sentiment import SentimentIntensityAnalyzer
from statistics import mean

sia = SentimentIntensityAnalyzer()
for each in data:
    scores = [sia.polarity_scores(sentence)["compound"] for sentence in nltk.sent_tokenize(each['content'])]
    print(mean(scores))
print("-------After Pre-processed Data-------")
for each in preprocessed_data:
    scores = [sia.polarity_scores(sentence)["compound"] for sentence in nltk.sent_tokenize(each)]
    print(mean(scores))

-0.15685625
-0.7474714285714286
-0.03154000000000001
0.18046666666666666
-0.360745
0.160475
-0.31245
-0.16955135135135135
0.03320000000000001
-0.2301764705882353
-0.319995
-0.1600921875
-0.389468
-0.03635
-------After Pre-processed Data-------
-0.68235
-0.9933
-0.304
0.5121142857142857
-0.9957
0.946
-0.6249
-0.4469
0.5859
-0.7179
-0.9949
-0.418925
-0.9894499999999999
-0.3142333333333333
